In [ ]:
pip install mido

     |████████████████████████████████| 61kB 7.2MB/s 


In [ ]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Bidirectional, LSTM, Reshape, RepeatVector, TimeDistributed
from keras.layers import BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
 
import mido # easy to use python MIDI library
import os # accessing directory structure
from __future__ import print_function, division
 
from mido import MidiFile, MidiTrack, Message
 
import matplotlib.pyplot as plt
 
import sys
 
import numpy as np
 
import os
 
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
paths = []
songs = []

for r, d, f in os.walk(r'/content/drive/MyDrive/Dataset/midi/blues'):
    for file in f:
        if '.mid' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    mid = MidiFile(path, type = 1)
    songs.append(mid)
del paths

In [ ]:
print(songs[0])

<midi file '/content/drive/MyDrive/Dataset/midi/blues/23_hours_blues_hh.mid' type 1, 1 tracks, 250 messages>


In [ ]:
notes = []
dataset = []
chunk = []

for i in range(len(songs)):
    for msg in songs[i]:
        if not msg.is_meta:
            if (msg.type == 'note_on'):
                notes.append(msg.note)
    for i in range(1, len(notes)):
        chunk.append(notes[i])
        if (i % 64 == 0):
            dataset.append(chunk)
            chunk = []
    chunk = []
    notes = []
del chunk
del notes

In [ ]:
print(dataset[0])

[49, 37, 44, 44, 49, 44, 42, 37, 42, 37, 37, 44, 44, 49, 49, 46, 44, 44, 39, 42, 37, 37, 44, 44, 49, 44, 49, 44, 42, 49, 37, 44, 44, 49, 44, 42, 37, 42, 37, 37, 44, 44, 49, 49, 46, 44, 44, 39, 42, 37, 37, 44, 44, 49, 44, 37, 44, 44, 42, 37, 37, 44, 44, 49]


In [ ]:
dataset = np.array(dataset)
dataset.shape

(530, 64)

In [ ]:
def load_data():
    x_train = dataset
    x_train = x_train.reshape(530,8,8)
    return x_train

In [ ]:
class LSTMGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 4
        self.img_cols = 4
        self.img_shape = (self.img_rows, self.img_cols)
        self.latent_dim = 16

        optimizer = Adam(0.0001, 0.4)

        # Build the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates song
        z = Input(shape=(4,4))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()
        model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(4, 4)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128, return_sequences=True)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128)))
        model.add(LeakyReLU(alpha=0.2))
        #specifying output to have 40 timesteps
        model.add(RepeatVector(64))
        #specifying 1 feature as the output
        model.add(Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.3))   
        model.add(TimeDistributed(Dense(128)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(128)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(1)))
        model.add(LeakyReLU(alpha=0.2))
        model.summary()

        noise = Input(shape=(4,4))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Bidirectional(LSTM(128, activation = 'relu', return_sequences=True), input_shape=(64, 1)))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Bidirectional(LSTM(128, activation = 'relu')))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(RepeatVector(4))
        model.add(TimeDistributed(Dense(128, activation = 'sigmoid')))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(128, activation = 'relu')))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.4))
        model.add(TimeDistributed(Dense(1, activation = 'linear')))
        model.summary()

        img = Input(shape=(64,1))
        validity = model(img)

        return Model(img, validity)
    

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        X_train = load_data()

        # Rescale 0 to 1
        X_train = X_train / 128

        # Adversarial ground truths
        valid = np.ones((batch_size,1,1))
        fake = np.zeros((batch_size,1,1))

        for epoch in range(epochs):
            #  Train Discriminator
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
            imgs = np.array(imgs)
            imgs = imgs.reshape(len(imgs),64,1)

            # Sample noise and generate a batch of new songs
            noise = np.random.normal(0, 1, (batch_size,4,4))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid)

            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            if epoch % save_interval == 0:
                self.generator.save("/content/drive/MyDrive/AI Project/Models/LSTM_generator.h5")

In [ ]:
lstmgan = LSTMGAN()
lstmgan.train(epochs=1000, batch_size=20, save_interval=100)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_8 (Bidirection (None, 64, 256)           133120    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 64, 256)           0         
_________________________________________________________________
bidirectional_9 (Bidirection (None, 256)               394240    
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 128)           

In [ ]:
from keras.models import load_model
model = load_model(r'/content/drive/MyDrive/AI Project/Models/5000_Epoch.h5')

In [ ]:
random = np.random.normal(.000005,.00001,(1,4,4))

In [ ]:
predict = model.predict(random)

In [ ]:
#normalization
predict = predict * 128

In [ ]:
print(predict)
predict.shape

[[[ 73.38678 ]
  [ 76.2939  ]
  [ 79.29709 ]
  [ 82.42817 ]
  [ 85.72317 ]
  [ 89.215225]
  [ 92.8423  ]
  [ 96.57019 ]
  [100.54338 ]
  [104.70265 ]
  [109.39954 ]
  [114.313576]
  [119.53083 ]
  [124.88176 ]
  [130.35994 ]
  [135.8582  ]
  [141.30405 ]
  [146.64664 ]
  [151.79082 ]
  [156.65288 ]
  [161.13487 ]
  [165.24202 ]
  [168.9553  ]
  [172.23106 ]
  [175.11234 ]
  [177.62685 ]
  [179.80775 ]
  [181.68823 ]
  [183.30241 ]
  [184.68158 ]
  [185.8543  ]
  [186.84535 ]
  [187.67885 ]
  [188.37454 ]
  [188.94955 ]
  [189.41776 ]
  [189.78482 ]
  [190.06233 ]
  [190.25911 ]
  [190.37846 ]
  [190.4212  ]
  [190.38686 ]
  [190.27182 ]
  [190.07002 ]
  [189.7725  ]
  [189.36801 ]
  [188.8365  ]
  [188.17009 ]
  [187.34624 ]
  [186.31696 ]
  [185.04268 ]
  [183.47372 ]
  [181.52208 ]
  [179.11118 ]
  [176.13443 ]
  [172.60905 ]
  [168.64453 ]
  [163.88808 ]
  [158.28294 ]
  [152.10538 ]
  [145.02632 ]
  [137.562   ]
  [130.2659  ]
  [125.58556 ]]]


(1, 64, 1)

In [ ]:
midler = MidiFile()
track = MidiTrack()
midler.tracks.append(track)
track.append(Message('program_change', program=2, time=0))
for x in range(64):
    track.append(Message('note_on', note=int(predict[0][x][0]), velocity=64, time=20))
    track.append(Message('note_off', note=int(predict[0][x][0]), velocity=64, time=20))
    midler.save('/content/drive/MyDrive/AI Project/Result/new_song.mid')